# Week 10 Quiz

## Lili Tan - lt2846

### Instructions

Replace the Name and UNI in cell above and the notebook name

Replace all '____' below using the instructions provided.

When completed, 
 - make sure you've replaced Name and UNI in the first cell and filename (eg: week_10_quiz-hw2592)
 - Click the github link shared on canvas for quiz10 to generate your own repository of quiz10
 - Click your own repository of quiz10 to commit your answers to github

In this quiz, we're going to load documents from 2 categories (space, cars) in the 20newsgroups dataset.

The goal is to train a classifier that classifies documents into these 2 categories based on a term frequency representation of the documents.

We will then calculate mean cross-validaion accuracy of a RandomForestClassifier using this transformation.

### Setup Environment

In [1]:
import numpy as np
import pandas as pd

### Load the Dataset

In [2]:
# Import fetch_20newsgroups from sklearn.datasets
from sklearn.datasets import fetch_20newsgroups 

# Load the dataset using fetch_20newsgroups().
#   Only fetch the two categories of interest using categories=['sci.space','rec.autos']
# Store in the result into newsgroups
newsgroups = fetch_20newsgroups(categories=['sci.space','rec.autos'])

# Store the newsgroups.data as docs, newsgroups.target as y and newsgroups.target_names as y_names
docs = newsgroups.data
y = newsgroups.target
y_names = newsgroups.target_names

# Print the number of observations by printing the length of docs
print(len(docs))

1187


In [3]:
# Print the text of the first document in docs.
# Note: try printing both with and without the print() statement
#  with the print statement, linebreaks are parsed,
#  without, linebreaks are printed as excape characters
print(docs[0])
docs[0]

From: prb@access.digex.com (Pat)
Subject: Re: Proton/Centaur?
Organization: Express Access Online Communications USA
Lines: 15
NNTP-Posting-Host: access.digex.net


Well thank you dennis for your as usual highly detailed and informative 
posting.   

The question i have about the proton, is  could it be  handled at
one of KSC's spare pads, without major  malfunction,  or could it be
handled at kourou  or Vandenberg?   

Now if it uses storables,  then  how long would it take for the russians
to equip something at cape york?

If  Proton were launched from a western site,  how would it compare to the
T4/centaur?   As i see it, it should lift  very close to the T4.

pat



"From: prb@access.digex.com (Pat)\nSubject: Re: Proton/Centaur?\nOrganization: Express Access Online Communications USA\nLines: 15\nNNTP-Posting-Host: access.digex.net\n\n\nWell thank you dennis for your as usual highly detailed and informative \nposting.   \n\nThe question i have about the proton, is  could it be  handled at\none of KSC's spare pads, without major  malfunction,  or could it be\nhandled at kourou  or Vandenberg?   \n\nNow if it uses storables,  then  how long would it take for the russians\nto equip something at cape york?\n\nIf  Proton were launched from a western site,  how would it compare to the\nT4/centaur?   As i see it, it should lift  very close to the T4.\n\npat\n"

In [4]:
# Print the target value of the first document in y.
print(y[0])

1


In [5]:
# Print the target_name of the first document using y_names and y.
print(y_names[y[0]])

sci.space


### Use CountVectorizer to Convert To TF

In [6]:
# Import CountVectorizer from sklearn
from sklearn.feature_extraction.text import CountVectorizer

# Initialize a CountVectorizer object. It should
#   lowercase all text, 
#   include both unigrams and bigrams
#   exclude terms that occur in fewer than 10 documents
#   exclude terms that occur in more than 95% of documents
# Store as cvect
cvect = CountVectorizer(lowercase=True,
                        ngram_range=(1,2), 
                        min_df=10, 
                        max_df=0.95)

# Fit cvect on docs and transform docs into their term frequency representation.
# Store as X_tf
X_tf = cvect.fit_transform(docs)

# Print the shape of X_tf. 
# The number of rows should match the number of documents 
#    and the number of columns should be in the thousands
X_tf.shape

(1187, 5893)

In [7]:
# Print out the last 5 terms in the learned vocabulary in cvect
#   using .get_feature_names() which returns a list of terms corresponding
#   to the order of the columns in X_tf
# They should all be related to zoos
cvect.get_feature_names()[-5:]

['zoo', 'zoo toronto', 'zoology', 'zoology kipling', 'zoology lines']

In [8]:
# The stopwords learned by cvect are stored as a set in cvect.stop_words_
# We'd like to print out a small subset of these terms.
# One way to get a subset of a set is to treat it as a list.
# First, convert the stop_words_ set to a list.
# Store as stop_words_list
stop_words_list = list(cvect.stop_words_)

# Print out the first 5 elements in stop_words_list.
# Note that, since a set is unordered, 
#    there is no meaning to the ordering of these terms and they may vary over runs.
stop_words_list[0:5]

['admit your',
 'jensen interceptors',
 'publishers starcat',
 'far too',
 'organization wilfrid']

### Calculate Mean CV Accuracy Using RandomForestClassifier

In [9]:
# Import cross_val_score from sklearn
from sklearn.model_selection import cross_val_score

# Import RandomForestClassifier from sklearn
from sklearn.ensemble import RandomForestClassifier

# Get a set of 5-fold CV scores using
#    a RandomForestClassifier with 50 trees and n_jobs=-1
#    and the full dataset X_tf and y
# Store as cv_scores
scores = cross_val_score(RandomForestClassifier(n_estimators=50,n_jobs=-1), X_tf, y, cv=5)

# Print the mean of these cv_scores. The mean accuracy should be above .9
scores.mean()

0.9629401127539623

### Optional: Find Important Features

In [10]:
# CountVectorizer stores the feature names (terms in the vocabulary) in two ways:
#  1. as a dictionary of term:colum_index pairs, accessed via cvect.vocabulary_
#  2. as a list of terms, in column index order, accessed via cvect.get_feature_names()
#
# We can get the indices of the most important features by retraining a RandomForestClassifier on X_tf,y
#  and accessing .feature_importances_
#
# Using some combination of the above data-structures, 
#  print out the top 10 terms in the vocabulary
#  ranked by the feature importances learned by a RandomForestClassifier with 50 trees
# 
# The terms you find will likely not be surprising given the document categories.
rf = RandomForestClassifier(n_estimators=50,random_state=123).fit(X_tf,y)
feature_importances = pd.DataFrame({'features':cvect.get_feature_names(),
                                    'feature_importance':rf.feature_importances_})
feature_importances.sort_values('feature_importance',ascending=False)[:10]

,features,feature_importance
1011,car,0.055405
1039,cars,0.031387
3196,moon,0.021729
4463,space,0.019814
3278,nasa,0.016829
3611,orbit,0.015197
1601,earth,0.011317
3873,prb access,0.010607
2804,launch,0.010206
4891,the moon,0.009171
